In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, pathpatch_2d_to_3d
import scipy.spatial as ss

from dosipy.utils.dataloader import load_antenna_el_properties
from dosipy.utils.viz import set_axes_equal, fig_config, save_fig

from utils import *

%config InlineBackend.figure_format = 'retina'

# input data

In [ ]:
# frequency
f = 5e9

# separation distance
h = -2.5 / 1000

# averaging surface
if f < 30e9:
    edge_length = 0.02
else:
    edge_length = 0.02
A = edge_length ** 2
target_area_origin = (-edge_length/2, -edge_length/2)

# source
data = load_antenna_el_properties(f)
xs = data.x.to_numpy()
xs -= xs.max() / 2
ys = np.zeros_like(xs) + h
zs = np.zeros_like(xs)
Is = np.abs(data.ireal.to_numpy() + 1j * data.iimag.to_numpy())

# planar target - reference evaluation surface

In [ ]:
N = 7
x = np.linspace(-edge_length/2, edge_length/2, N)
y = 0.0
z = np.linspace(-edge_length/2, edge_length/2, N)
Xt, Zt = np.meshgrid(x, z)
xt_pln = Xt.ravel()
yt_pln = np.zeros_like(xt_pln)
zt_pln = Zt.ravel()

nx_pln = 0
ny_pln = -1
nz_pln = 0

In [ ]:
fig_config(latex=True)
fig = plt.figure()
ax = plt.axes(projection ='3d')
plane = Rectangle(target_area_origin, width=edge_length, height=edge_length,
                  ec='k', ls='-', fc='gray', alpha=0.5)
ax.add_patch(plane)
pathpatch_2d_to_3d(plane, z=y, zdir='y')
ax.scatter(xt_pln, yt_pln, zt_pln, s=7, color='k', depthshade=True)
ax.quiver(xt_pln, yt_pln, zt_pln, nx_pln, ny_pln, nz_pln, normalize=True,
          arrow_length_ratio=0.33, lw=1, length=np.abs(h)/3, color='k')
ax.plot(xs[:24], ys[:24], zs[:24], 'k-')
ax.plot(xs[28:], ys[28:], zs[28:], 'k-')
ax.set_box_aspect([1, 1, 1])
ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
       xticks=[xt_pln.min(), 0.0, xt_pln.max()],
       yticks=[h, h/2, 0.0],
       zticks=[zt_pln.min(), 0.0, zt_pln.max()],
       xticklabels=[int(xt_pln.min()*1000), 0, int(xt_pln.max()*1000)],
       yticklabels=['$d$', '$d/2$', 0],
       zticklabels=[int(zt_pln.min()*1000), 0, int(zt_pln.max()*1000)],
       xlim=[xs.min()*1.1, xs.max()*1.1],
       ylim=[h*1.1, 0.0],
       zlim=[xs.min()*1.1, xs.max()*1.1],
      )
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 10
ax.view_init(25, -45)
fig.tight_layout()

# fname = 'evaluation_plane'
# save_fig(fig, os.path.join('figures', fname), formats=['pdf', 'png'])

# spherical target - evaluation surface

In [ ]:
r = 0.076
alpha = 2 * np.arcsin(edge_length/2/r)  # compute angle from secant
N = 7
theta = np.linspace(np.pi/2 - alpha/2, np.pi/2 + alpha/2, N)
phi = np.linspace(np.pi-alpha/2, np.pi+alpha/2, N)
Theta, Phi = np.meshgrid(theta, phi)
yt_sph, xt_sph, zt_sph = sph2cart(r, Theta.ravel(), Phi.ravel())
yt_sph -= yt_sph.min()

ny_sph, nx_sph, nz_sph = sph_normals(r, Theta.ravel(), Phi.ravel())

In [ ]:
fig_config(latex=True, scaler=1)
fig = plt.figure()
ax = plt.axes(projection ='3d')
plane = Rectangle(target_area_origin, width=edge_length, height=edge_length,
                  ec='k', ls='-', fc='gray', alpha=0.3)
ax.add_patch(plane)
pathpatch_2d_to_3d(plane, z=y, zdir='y')
ax.scatter(xt_sph, yt_sph, zt_sph, s=7, color='k', depthshade=True)
ax.quiver(xt_sph, yt_sph, zt_sph, nx_sph, ny_sph, nz_sph, normalize=True,
          arrow_length_ratio=0.33, lw=1, length=np.abs(h)/2.5, color='k')
ax.plot(xs[:24], ys[:24], zs[:24], 'k-')
ax.plot(xs[28:], ys[28:], zs[28:], 'k-')
ax.set_box_aspect([1, 1, 1])
ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
       xticks=[xt_pln.min(), 0.0, xt_pln.max()],
       yticks=[h, h/2, 0.0],
       zticks=[zt_pln.min(), 0.0, zt_pln.max()],
       xticklabels=[int(xt_pln.min()*1000), 0, int(xt_pln.max()*1000)],
       yticklabels=['$d$', '$d/2$', 0],
       zticklabels=[int(zt_pln.min()*1000), 0, int(zt_pln.max()*1000)],
       xlim=[xs.min()*1.1, xs.max()*1.1],
       ylim=[h*1.1, yt_sph.max()],
       zlim=[xs.min()*1.1, xs.max()*1.1],
      )
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 10
ax.view_init(25, -50)
fig.tight_layout()

# fname = 'evaluation_sphere'
# save_fig(fig, os.path.join('figures', fname), formats=['pdf', 'png'])

# spherical target - model

In [ ]:
theta = np.linspace(0, 2 * np.pi, 17)
phi = np.linspace(0, np.pi, 9)
Theta, Phi = np.meshgrid(theta, phi)

# model of an average adult
r_adult = 0.076
xt_adult, yt_adult, zt_adult = sph2cart(r_adult, Theta.ravel(), Phi.ravel())
xyz_adult = np.c_[xt_adult, yt_adult, zt_adult]
hull_adult = ss.ConvexHull(xyz_adult)

# model of an average child
r_child = 0.057
xt_child, yt_child, zt_child = sph2cart(r_child, Theta.ravel(), Phi.ravel())
xyz_child = np.c_[xt_child, yt_child, zt_child]
hull_child = ss.ConvexHull(xyz_child)

In [ ]:
# adult
fig_config(latex=True)
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.scatter(xt_adult, yt_adult, zt_adult, s=7, marker='.', ec='k', c='None')
hull_triangle_coords = hull_adult.points[hull_adult.simplices]
triangles = Poly3DCollection(hull_triangle_coords, color='r', ec='k', lw=0.1, alpha=0.75)
ax.add_collection3d(triangles)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
       xticks=[-r_adult, 0, r_adult],
       yticks=[-r_adult, 0, r_adult],
       zticks=[-r_adult, 0, r_adult],
       xticklabels=[-int(r_adult * 1000), 0, int(r_adult * 1000)],
       yticklabels=[-int(r_adult * 1000), 0, int(r_adult * 1000)],
       zticklabels=[-int(r_adult * 1000), 0, int(r_adult * 1000)],
       xlim=[-r_adult * 1.2, r_adult * 1.2],
       ylim=[-r_adult * 1.2, r_adult * 1.2],
       zlim=[-r_adult * 1.2, r_adult * 1.2],
      )
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 7
ax.yaxis.labelpad = 7
ax.zaxis.labelpad = 7
ax.view_init(25, -45)
fig.tight_layout();

# fname = 'head_adult'
# save_fig(fig, os.path.join('figures', fname), formats=['pdf', 'png'])

In [ ]:
# child
fig_config(latex=True)
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.scatter(xt_child, yt_child, zt_child, s=7, marker='.', ec='k', c='None')
hull_triangle_coords = hull_child.points[hull_adult.simplices]
triangles = Poly3DCollection(hull_triangle_coords, color='b', ec='k', lw=0.1, alpha=0.5)
ax.add_collection3d(triangles)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
       xticks=[-r_child, 0, r_child],
       yticks=[-r_child, 0, r_child],
       zticks=[-r_child, 0, r_child],
       xticklabels=[-int(r_child * 1000), 0, int(r_child * 1000)],
       yticklabels=[-int(r_child * 1000), 0, int(r_child * 1000)],
       zticklabels=[-int(r_child * 1000), 0, int(r_child * 1000)],
       xlim=[-r_adult * 1.2, r_adult * 1.2],
       ylim=[-r_adult * 1.2, r_adult * 1.2],
       zlim=[-r_adult * 1.2, r_adult * 1.2],
      )
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 7
ax.yaxis.labelpad = 7
ax.zaxis.labelpad = 7
ax.view_init(25, -45)

# fname = 'head_child'
# save_fig(fig, os.path.join('figures', fname), formats=['pdf', 'png'])

# min-max curvature

In [ ]:
N = 101

r_min = 1.8 / 100
alpha_min = 2 * np.arcsin(edge_length/2/r_min)
theta_min = np.linspace(np.pi/2 - alpha_min/2, np.pi/2 + alpha_min/2, N)
phi_min = np.linspace(np.pi-alpha_min/2, np.pi+alpha_min/2, N)
Theta_min, Phi_min = np.meshgrid(theta_min, phi_min)
yt_min, xt_min, zt_min = sph2cart(r_min, Theta_min.ravel(), Phi_min.ravel())
yt_min -= yt_min.min()

r_max = 18 / 100
alpha_max = 2 * np.arcsin(edge_length/2/r_max)
theta_max = np.linspace(np.pi/2 - alpha_max/2, np.pi/2 + alpha_max/2, N)
phi_max = np.linspace(np.pi-alpha_max/2, np.pi+alpha_max/2, N)
Theta_max, Phi_max = np.meshgrid(theta_max, phi_max)
yt_max, xt_max, zt_max = sph2cart(r_max, Theta_max.ravel(), Phi_max.ravel())
yt_max -= yt_max.min()

In [ ]:
fig_config(latex=True, scaler=1)
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.scatter(xt_max, yt_max, zt_max, s=1, color='r', alpha=0.1)
ax.plot([], [], [], 'rs', markeredgecolor='r', ms=9,
        label=r'$r_\text{max}$')
ax.scatter(xt_min, yt_min, zt_min, s=1, color='b', alpha=0.1)
ax.plot([], [], [], 'bs', markeredgecolor='b', ms=9,
        label=r'$r_\text{min}$')
icntr = int(xt_min.size/2)
z_ref = xs.min()*1.1
ax.plot(xt_min[icntr], yt_min[icntr], zt_min[icntr], 'kx',
        markeredgecolor='k', markersize=7, markeredgewidth=1.5, zorder=4,
        label='$(0, 0, 0)$')
ax.plot(xt_min[icntr], yt_min[icntr], 'k.',
        markeredgecolor='k', markersize=5, markeredgewidth=1.5,
        zdir='z', zs=z_ref, zorder=5)
ax.plot(np.ones(50)*xt_min[icntr], np.linspace(z_ref, zt_min[icntr]), 'k--',
        lw=1, zdir='y', zs=yt_min.min(), zorder=6)
ax.set_box_aspect([1, 1, 1])
ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
       xticks=[xt_min.min(), 0.0, xt_min.max()],
       yticks=[yt_min.min(), (yt_min.min()+yt_min.max())/2, yt_min.max()],
       zticks=[zt_min.min(), 0.0, zt_min.max()],
       xticklabels=[round(xt_min.min()*1000), 0, round(xt_min.max()*1000)],
       yticklabels=[round(yt_min.min()*1000),
                    round((yt_min.min()+yt_min.max())/2*1000),
                    round(yt_min.max()*1000)],
       zticklabels=[round(zt_min.min()*1000), 0, round(zt_min.max()*1000)],
       xlim=[xs.min()*1.1, xs.max()*1.1],
       ylim=[yt_min.min()-yt_min.max()*0.1, yt_min.max()*1.1],
       zlim=[xs.min()*1.1, xs.max()*1.1],
      )
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 10
ax.view_init(20, -60)
fig.legend(bbox_to_anchor=(0.25, 0.675), ncol=1)
fig.tight_layout()

# fname = 'evaluation_min_max'
# save_fig(fig, os.path.join('figures', fname), formats=['png'])